In [1]:
import pandas as pd


### Задание 1.
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [2]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
df_grouped = df.groupby('userId').count().reset_index()
df_grouped.head()

,userId,movieId,rating,timestamp
0,1,20,20,20
1,2,76,76,76
2,3,51,51,51
3,4,204,204,204
4,5,100,100,100


In [4]:
df_filtered = df_grouped[df_grouped['rating']>=100]
df_filtered.head()

,userId,movieId,rating,timestamp
3,4,204,204,204
4,5,100,100,100
7,8,116,116,116
14,15,1700,1700,1700
16,17,363,363,363


In [6]:
fans_user_ids = df_filtered['userId'].tolist()

In [8]:
df100 = df[ df['userId'].isin(fans_user_ids) ]
df100.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [15]:
lifetime = df100.groupby('userId').max()['timestamp'] - df100.groupby('userId').min()['timestamp']
lifetime.head()

userId
4        203560
5          2101
8         85187
15    471393496
17         8053
Name: timestamp, dtype: int64

In [18]:
# среднее время жизни пользователей в днях
lifetime.mean()/ 24 / 3600

455.2285713719898

#### Задание 1. Решение с помощью сводных таблиц

In [20]:
df_min = df.pivot_table(index='userId', values = 'timestamp', aggfunc = 'min', fill_value = 0)
df_min.head()

,timestamp
userId,
1,1260759108
2,835355395
3,1298861589
4,949778714
5,1163373044


In [22]:
df_max = df.pivot_table(index='userId', values = 'timestamp', aggfunc = 'max', fill_value = 0)
df_max.head()

,timestamp
userId,
1,1260759205
2,835356246
3,1298932787
4,949982274
5,1163375145


In [23]:
df_cnt = df.pivot_table(index='userId', values = 'timestamp', aggfunc = 'count', fill_value = 0)
df_cnt.head()

,timestamp
userId,
1,20
2,76
3,51
4,204
5,100


In [27]:
df_users = df_max.merge(df_min, how = 'left', on = 'userId').merge(df_cnt, how = 'left', on = 'userId')
df_users.rename(columns={'timestamp': 'rates'}, inplace=True)
df_users.head()

,timestamp_x,timestamp_y,rates
userId,,,
1,1260759205,1260759108,20
2,835356246,835355395,76
3,1298932787,1298861589,51
4,949982274,949778714,204
5,1163375145,1163373044,100


In [28]:
df_users = df_users[df_users.rates >= 100]

In [29]:
df_users['lifetime'] = df_users.timestamp_x - df_users.timestamp_y
df_users.head()

,timestamp_x,timestamp_y,rates,lifetime
userId,,,,
4,949982274,949778714,204,203560
5,1163375145,1163373044,100,2101
8,1154474527,1154389340,116,85187
15,1469330735,997937239,1700,471393496
17,1127476640,1127468587,363,8053


In [30]:
df_users['lifetime'].mean()/ 24 / 3600

455.2285713719898

### Задание 2.
Дана статистика услуг перевозок клиентов компании по типам. Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

    rzd - железнодорожные перевозки
    auto - автомобильные перевозки
    air - воздушные перевозки
    client_base - адреса клиентов


In [31]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [32]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [33]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [34]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [35]:
# таблица с тремя типами выручки для каждого client_id без указания адреса клиента
transport = rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id')
transport

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [36]:
# таблица с тремя типами выручки с адресом клиента
client_trans = transport.merge(client_base, how='left', on='client_id')
client_trans

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


### Задание 3.
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

Дайте комментарии в свободной форме. Никакого точного ответа у задачи нет (при такой постановке)

Вероятно, данных о визитах и покупках меньше, чем данных о координатах. Это затрудняет прямое сопоставлени через user_id. 
Скорее всего, данные о координатах связаны со временем, когда приложение их передавало.
Визиты и покупки пользователей тоже могут иметь привязку ко времени в момент входа в приложение и в момент покупки. То есть можно сопоставлять визиты и координаты через user_id и ближайшее время. 
 